# Яндекс.Музыка

**Цель исследования** — проверить три гипотезы сравните поведение пользователей двух столиц на данных Яндекс.Музыки:
1. Активность пользователей зависит от дня недели. Причём в Москве и Петербурге это проявляется по-разному.
2. В понедельник утром в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города. 
3. Москва и Петербург предпочитают разные жанры музыки. В Москве чаще слушают поп-музыку, в Петербурге — русский рэп.

**Ход исследования**  

Исследование пройдёт в три этапа:
 1. Обзор данных.
 2. Предобработка данных.
 3. Проверка гипотез.



## Обзор данных

In [1]:
import pandas as pd

In [2]:
local_path = './yandex_music_project.csv'
server_path = '/datasets/yandex_music_project.csv'
try:
    df = pd.read_csv(local_path)
except:
    df = pd.read_csv(server_path)

Для ознакомления с DF выведем его первые 10 строк:

In [3]:
df.head(10)

,userID,Track,artist,genre,City,time,Day
0,FFB692EC,Kamigata To Boots,The Mass Missile,rock,Saint-Petersburg,20:28:33,Wednesday
1,55204538,Delayed Because of Accident,Andreas Rönnberg,rock,Moscow,14:07:09,Friday
2,20EC38,Funiculì funiculà,Mario Lanza,pop,Saint-Petersburg,20:58:07,Wednesday
3,A3DD03C9,Dragons in the Sunset,Fire + Ice,folk,Saint-Petersburg,08:37:09,Monday
4,E2DC1FAE,Soul People,Space Echo,dance,Moscow,08:34:34,Monday
5,842029A1,Преданная,IMPERVTOR,rusrap,Saint-Petersburg,13:09:41,Friday
6,4CB90AA5,True,Roman Messer,dance,Moscow,13:00:07,Wednesday
7,F03E1C1F,Feeling This Way,Polina Griffith,dance,Moscow,20:47:49,Wednesday
8,8FA1D3BE,И вновь продолжается бой,NaN,ruspop,Moscow,09:17:40,Friday
9,E772D5C0,Pessimist,NaN,dance,Saint-Petersburg,21:20:49,Wednesday


Получим общую информацию о таблице:

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 65079 entries, 0 to 65078
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0     userID  65079 non-null  object
 1   Track     63848 non-null  object
 2   artist    57876 non-null  object
 3   genre     63881 non-null  object
 4     City    65079 non-null  object
 5   time      65079 non-null  object
 6   Day       65079 non-null  object
dtypes: object(7)
memory usage: 3.5+ MB


В названиях колонок видны три нарушения стиля:
1. строчные буквы сочетаются с прописными,
2. встречаются пробелы в начале строки названия,
3. названия колонок состоят из нескольких слов и в качестве разделителя использован пробел вместо _.  

Кроме того количество значений в столбцах различается. Значит, в данных есть пропущенные значения.

**Вывод**

В каждой строке таблицы — данные о прослушанном треке. Часть колонок описывает саму композицию: название, исполнителя и жанр. Остальные данные рассказывают о пользователе: из какого он города, когда он слушал музыку. 

Предварительно можно утверждать, что, данных достаточно для проверки гипотез. Но встречаются пропуски в данных, а в названиях колонок — расхождения с хорошим стилем.

## Предобработка данных
Исправим стиль в заголовках столбцов, исключим пропуски, а затем проверим данные на дубликаты.

### Стиль заголовков

In [5]:
df.columns

Index(['  userID', 'Track', 'artist', 'genre', '  City  ', 'time', 'Day'], dtype='object')

In [6]:
df = df.rename(columns= {
    '  userID': 'user_id', 
    'Track': 'track', 
    '  City  ': 'city', 
    'Day': 'day'})

In [7]:
df.columns

Index(['user_id', 'track', 'artist', 'genre', 'city', 'time', 'day'], dtype='object')

### Пропуски значений

In [8]:
df.isna().sum()

user_id       0
track      1231
artist     7203
genre      1198
city          0
time          0
day           0
dtype: int64

Пропуски есть в трех колонках: `track`, `artist` и `genre`. В первых двух пропуски не важны для решения поставленной задачи, а в третьей пропуски могут оказать влияние на конечный вывод. Для начала заменим пропуски во всех колонках значением `'unknown'`.

In [9]:
columns_to_replace = ['track', 'artist', 'genre']
for column in columns_to_replace:
    df[column] = df[column].fillna('unknown')

In [10]:
# проверим заполнение пропусков
df.isna().sum()

user_id    0
track      0
artist     0
genre      0
city       0
time       0
day        0
dtype: int64

### Дубликаты

In [11]:
df.duplicated().sum()

3826

In [12]:
# удаление явных дубликатов (с удалением старых индексов и формированием новых)
df = df.drop_duplicates().reset_index(drop=True)

In [13]:
# проверка на отсутствие дубликатов
df.duplicated().sum()

0

Далее проведем очистку от неявных дубликатов в колонке `genre`.

In [14]:
# получим список всех вариантов написания жанров
sorted(df['genre'].unique())

['acid',
 'acoustic',
 'action',
 'adult',
 'africa',
 'afrikaans',
 'alternative',
 'alternativepunk',
 'ambient',
 'americana',
 'animated',
 'anime',
 'arabesk',
 'arabic',
 'arena',
 'argentinetango',
 'art',
 'audiobook',
 'author',
 'avantgarde',
 'axé',
 'baile',
 'balkan',
 'beats',
 'bigroom',
 'black',
 'bluegrass',
 'blues',
 'bollywood',
 'bossa',
 'brazilian',
 'breakbeat',
 'breaks',
 'broadway',
 'cantautori',
 'cantopop',
 'canzone',
 'caribbean',
 'caucasian',
 'celtic',
 'chamber',
 'chanson',
 'children',
 'chill',
 'chinese',
 'choral',
 'christian',
 'christmas',
 'classical',
 'classicmetal',
 'club',
 'colombian',
 'comedy',
 'conjazz',
 'contemporary',
 'country',
 'cuban',
 'dance',
 'dancehall',
 'dancepop',
 'dark',
 'death',
 'deep',
 'deutschrock',
 'deutschspr',
 'dirty',
 'disco',
 'dnb',
 'documentary',
 'downbeat',
 'downtempo',
 'drum',
 'dub',
 'dubstep',
 'eastern',
 'easy',
 'electronic',
 'electropop',
 'emo',
 'entehno',
 'epicmetal',
 'estrada',


Визуальным осмотром полученного списка замечены неявные дубликаты названия `hiphop` - три варианта написания:
- hip,
- hop,
- hip-hop.

Чтобы очистить от них таблицу, используем собственную функцию `replace_wrong_genres()` с двумя параметрами: 
* `wrong_genres` — список дубликатов,
* `correct_genre` — строка с правильным значением.

In [15]:
def replace_wrong_genres(wrong_genres, correct_genre):
    for wrong_genre in wrong_genres:
        df['genre'] = df['genre'].replace(wrong_genre, correct_genre)

In [16]:
duplicates = ['hip', 'hop', 'hip-hop']
genre = 'hiphop'
replace_wrong_genres(duplicates, genre)

In [17]:
# проверка корректности отработки функции
sorted(df['genre'].unique())

['acid',
 'acoustic',
 'action',
 'adult',
 'africa',
 'afrikaans',
 'alternative',
 'alternativepunk',
 'ambient',
 'americana',
 'animated',
 'anime',
 'arabesk',
 'arabic',
 'arena',
 'argentinetango',
 'art',
 'audiobook',
 'author',
 'avantgarde',
 'axé',
 'baile',
 'balkan',
 'beats',
 'bigroom',
 'black',
 'bluegrass',
 'blues',
 'bollywood',
 'bossa',
 'brazilian',
 'breakbeat',
 'breaks',
 'broadway',
 'cantautori',
 'cantopop',
 'canzone',
 'caribbean',
 'caucasian',
 'celtic',
 'chamber',
 'chanson',
 'children',
 'chill',
 'chinese',
 'choral',
 'christian',
 'christmas',
 'classical',
 'classicmetal',
 'club',
 'colombian',
 'comedy',
 'conjazz',
 'contemporary',
 'country',
 'cuban',
 'dance',
 'dancehall',
 'dancepop',
 'dark',
 'death',
 'deep',
 'deutschrock',
 'deutschspr',
 'dirty',
 'disco',
 'dnb',
 'documentary',
 'downbeat',
 'downtempo',
 'drum',
 'dub',
 'dubstep',
 'eastern',
 'easy',
 'electronic',
 'electropop',
 'emo',
 'entehno',
 'epicmetal',
 'estrada',


**Выводы**

Исправлены недочеты в данных, выявленные на этапе предобработки:

- нарушения в стиле заголовков,
- пропущенные значения,
- дубликаты — явные и неявные.

## Проверка гипотез

### Сравнение поведения пользователей двух столиц

Первая гипотеза утверждает, что пользователи по-разному слушают музыку в Москве и Санкт-Петербурге. Проверим это предположение по данным о трёх днях недели — понедельнике, среде и пятнице. 

Для оценки активности пользователей в каждом городе сгруппируем данные по городу и посчитаем прослушивания в каждой группе.

In [18]:
df.pivot_table(index=['city'], aggfunc='count', values='user_id')

,user_id
city,
Moscow,42741
Saint-Petersburg,18512


В Москве прослушиваний больше, чем в Петербурге, однако нельзя с уверенностью сказать, что там слушают больше, скорее всего самих пользователей в Москве больше.

Далее просчитаем прослушивания в понедельник, среду и пятницу суммарно по двум городам.

In [19]:
df.groupby('day')['city'].count()

day
Friday       21840
Monday       21354
Wednesday    18059
Name: city, dtype: int64

В среднем пользователи из двух городов менее активны по средам. Но более объективно можно оценить эту информацию, если рассмотреть каждый город в отдельности.

In [20]:
def number_tracks(day, city):
    track_list = df[(df['city'] == city) & (df['day'] == day)]['user_id'].count()
    return track_list

In [21]:
city_day = {'Moscow':[], 'Saint-Petersburg':[]}
for city in city_day.keys():
    for day in ['Monday', 'Wednesday', 'Friday']:
        city_day[city].append(number_tracks(day, city))

In [22]:
data_city_day = pd.DataFrame.from_dict(
    city_day, orient='index', 
    columns=['monday', 'wednesday', 'friday'])
data_city_day

,monday,wednesday,friday
Moscow,15740,11056,15945
Saint-Petersburg,5614,7003,5895


**Выводы**

Данные показывают разницу поведения пользователей:

- В Москве пик прослушиваний приходится на понедельник и пятницу, а в среду заметен спад.
- В Петербурге, наоборот, больше слушают музыку по средам. Активность в понедельник и пятницу здесь почти в равной мере уступает среде.

Значит, данные говорят в пользу первой гипотезы.

### Музыка в начале и в конце недели

Согласно второй гипотезе, утром в понедельник в Москве преобладают одни жанры, а в Петербурге — другие. Так же и вечером пятницы преобладают разные жанры — в зависимости от города.

Создадим функцию `genre_weekday()` с четырьмя параметрами для определения предпочтений пользователей по жанрам в разное время суток:
* таблица (датафрейм) с данными,
* день недели,
* начальная временная метка в формате 'hh:mm', 
* последняя временная метка в формате 'hh:mm'.

In [23]:
def genre_weekday(table, day, time_start, time_end):
    genre_df = table[(table['day'] == day) & (table['time'] > time_start) & (table['time'] < time_end)]
    genre_df_count = genre_df.groupby('genre')['user_id'].count().sort_values(ascending=False)
    return genre_df_count.head(10)

Для анализа используем функцию `genre_weekday()` и получим данные для Москвы и Санкт-Петербурга в понедельник утром (с 7:00 до 11:00) и в пятницу вечером (с 17:00 до 23:00):

In [24]:
for city in ['Moscow','Saint-Petersburg']:
    for day in [['Monday', '07:00', '11:00'], ['Friday', '17:00', '23:00']]:
        print(f'{city, day}', '\n',genre_weekday(df.query('city == @city'), *day))
        print(''*30)

('Moscow', ['Monday', '07:00', '11:00']) 
 genre
pop            781
dance          549
electronic     480
rock           474
hiphop         286
ruspop         186
world          181
rusrap         175
alternative    164
unknown        161
Name: user_id, dtype: int64

('Moscow', ['Friday', '17:00', '23:00']) 
 genre
pop            713
rock           517
dance          495
electronic     482
hiphop         273
world          208
ruspop         170
alternative    163
classical      163
rusrap         142
Name: user_id, dtype: int64

('Saint-Petersburg', ['Monday', '07:00', '11:00']) 
 genre
pop            218
dance          182
rock           162
electronic     147
hiphop          80
ruspop          64
alternative     58
rusrap          55
jazz            44
classical       40
Name: user_id, dtype: int64

('Saint-Petersburg', ['Friday', '17:00', '23:00']) 
 genre
pop            256
electronic     216
rock           216
dance          210
hiphop          97
alternative     63
jazz         

**Выводы**

Если сравнить топ-10 жанров в понедельник утром, можно сделать такие выводы:

1. В Москве и Петербурге слушают похожую музыку. Единственное отличие — в московский рейтинг вошёл жанр “world”, а в петербургский — джаз и классика.

2. В Москве пропущенных значений оказалось так много, что значение `'unknown'` заняло десятое место среди самых популярных жанров. Значит, пропущенные значения занимают существенную долю в данных и угрожают достоверности исследования.

Вечер пятницы не меняет эту картину. Некоторые жанры поднимаются немного выше, другие спускаются, но в целом топ-10 остаётся тем же самым.

Таким образом, вторая гипотеза подтвердилась лишь частично:
* Пользователи слушают похожую музыку в начале недели и в конце.
* Разница между Москвой и Петербургом не слишком выражена. В Москве чаще слушают русскую популярную музыку, в Петербурге — джаз.

Однако пропуски в данных ставят под сомнение этот результат. В Москве их так много, что рейтинг топ-10 мог бы выглядеть иначе, если бы не утерянные  данные о жанрах.

### Жанровые предпочтения в Москве и Петербурге

Гипотеза: Петербург — столица рэпа, музыку этого жанра там слушают чаще, чем в Москве.  А Москва — город контрастов, в котором, тем не менее, преобладает поп-музыка.

In [25]:
for city in ['Moscow','Saint-Petersburg']:
    print(f'{city}', '\n', df.query('city == @city').pivot_table(
        'user_id', 'genre', aggfunc='count').sort_values(
        by='user_id', ascending=False).rename(columns={'user_id':'number'}).head(10))
    print(''*30)

Moscow 
              number
genre              
pop            5892
dance          4435
rock           3965
electronic     3786
hiphop         2096
classical      1616
world          1432
alternative    1379
ruspop         1372
rusrap         1161

Saint-Petersburg 
              number
genre              
pop            2431
dance          1932
rock           1879
electronic     1736
hiphop          960
alternative     649
classical       646
rusrap          564
ruspop          538
world           515



**Выводы**

Гипотеза частично подтвердилась:
* Поп-музыка — самый популярный жанр в Москве, как и предполагала гипотеза. Более того, в топ-10 жанров встречается близкий жанр — русская популярная музыка.
* Вопреки ожиданиям, рэп одинаково популярен в Москве и Петербурге. 


## Итоги исследования

По результатам проведенных расчетов можно сделать следующие выводы:

1. **Гипотеза 1**: *День недели по-разному влияет на активность пользователей в Москве и Петербурге*. 

Первая гипотеза полностью подтвердилась.

2. **Гипотеза 2**: *Музыкальные предпочтения не сильно меняются в течение недели — будь то Москва или Петербург.*  
Небольшие различия заметны в начале недели, по понедельникам:
* в Москве слушают музыку жанра “world”,
* в Петербурге — джаз и классику.

Таким образом, вторая гипотеза подтвердилась лишь отчасти. Этот результат мог оказаться иным, если бы не пропуски в данных.

3. **Гипотеза 3:** *Во вкусах пользователей Москвы и Петербурга больше общего, чем различий.*  
Вопреки ожиданиям, предпочтения жанров в Петербурге напоминают московские.

Третья гипотеза подтвердилась. Если различия в предпочтениях и существуют, на основной массе пользователей они незаметны.